In [1]:
import cv2
import mediapipe as mp
import random
import time

# === Setup MediaPipe ===
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1)

# === Mapping ===
choices = ['Rock', 'Paper', 'Scissors']
score = {'You': 0, 'Computer': 0}

def detect_hand_sign(landmarks):
    fingers = []

    # Thumb (compare tip to base)
    if landmarks[4].x < landmarks[3].x:
        fingers.append(1)
    else:
        fingers.append(0)

    # Index, Middle, Ring, Pinky fingers
    for tip in [8, 12, 16, 20]:
        if landmarks[tip].y < landmarks[tip - 2].y:
            fingers.append(1)
        else:
            fingers.append(0)

    # Match gesture
    if fingers == [0, 0, 0, 0, 0]:
        return "Rock"
    elif fingers == [1, 1, 1, 1, 1]:
        return "Paper"
    elif fingers == [0, 1, 1, 0, 0]:
        return "Scissors"
    else:
        return "Unknown"

def decide_winner(player, computer):
    if player == computer:
        return "Draw"
    if (player == "Rock" and computer == "Scissors") or \
       (player == "Scissors" and computer == "Paper") or \
       (player == "Paper" and computer == "Rock"):
        score['You'] += 1
        return "You Win!"
    else:
        score['Computer'] += 1
        return "Computer Wins!"

# === Start Webcam ===
cap = cv2.VideoCapture(0)
start_time = time.time()
play = False
result = ""
your_move = ""
computer_move = ""

print("✊✋✌️ Rock Paper Scissors AI Game Started")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            your_move = detect_hand_sign(hand_landmarks.landmark)

    # Countdown before play
    elapsed = time.time() - start_time
    if elapsed > 5:
        play = True
        start_time = time.time()  # Reset timer
        computer_move = random.choice(choices)
        if your_move != "Unknown":
            result = decide_winner(your_move, computer_move)
        else:
            result = "Could not detect move."

    # === Display Info ===
    cv2.putText(frame, f"Your Move: {your_move}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
    cv2.putText(frame, f"Computer: {computer_move}", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    cv2.putText(frame, f"Result: {result}", (10, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 3)
    cv2.putText(frame, f"Score - You: {score['You']} | Computer: {score['Computer']}", (10, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
    cv2.putText(frame, f"Showing Result every 5 sec", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

    cv2.imshow("Rock Paper Scissors - AI", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
      0156+988502  break

# === Cleanup ===
cap.release()
cv2.destroyAllWindows()

✊✋✌️ Rock Paper Scissors AI Game Started
